# Física Nuclear y de Partículas #

## Tema 2. La interacción nuclear.  ##


César Fernández Ramírez\
Departamento de Física Interdisciplinar\
Universidad Nacional de Eduación a Distancia

Versión: Junio 2024

Contenidos:

- 



In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import pandas as pd
import matplotlib.ticker as ticker

jpac_blue   = "#1F77B4"; jpac_red    = "#D61D28";
jpac_green  = "#2CA02C"; jpac_orange = "#FF7F0E";
jpac_purple = "#9467BD"; jpac_brown  = "#8C564B";
jpac_pink   = "#E377C2"; jpac_gold   = "#BCBD22";
jpac_aqua   = "#17BECF"; jpac_grey   = "#7F7F7F";

jpac_color = [jpac_blue, jpac_red, jpac_green, 
              jpac_orange, jpac_purple, jpac_brown,
              jpac_pink, jpac_gold, jpac_aqua, jpac_grey, 'black' ];

jpac_axes = jpac_color[10]
dashes = 60*'-'

# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all")
df = df[pd.to_numeric(df['binding'],errors='coerce').notna()]
df['binding'] /= 1000
df['A'] = df['n']+df['z']

uma = 931.4941024/1000000/1000

def yukawa(k,la,r):
    return -k*np.exp(-r/la)/r
def yukawaeff(k,la,l,m,r):
    return yukawa(k,la,r) + l*(l+1)/(2*m*r**2)


Deuterón

In [89]:
deuteron = df.loc[ ((df['z']==1) & (df['n']==1))] 
print(dashes+dashes)
print('Propiedades del deuterón:')
print('A=',deuteron['A'].values[0],';','Z=',deuteron['z'].values[0],'(',deuteron['symbol'].values[0],');','N=',deuteron['n'].values[0])
print('Año de descubrimiento: ', int(deuteron['discovery'].values[0]))
print('Vida media: ', deuteron['half_life'].values[0])
print('Abundancia: ', deuteron['abundance'].values[0],'%')
print('Radio de carga: ', deuteron['radius'].values[0],'+/-',deuteron['unc_r'].values[0],'fm')
print('Energía de ligadura: ', deuteron['binding'].values[0]*deuteron['A'].values[0],'+/-',deuteron['unc_ba'].values[0]*deuteron['A'].values[0],'MeV')
print('Masa: ', deuteron['atomic_mass'].values[0]/1000,'+/-',deuteron['unc_am'].values[0]/1000,'u.m.a.',';',deuteron['atomic_mass'].values[0]*uma,'GeV')
print('Dipolo magnético: ', deuteron['magnetic_dipole'].values[0],'magnetones de Bohr')
print('Cuadrupolo eléctrico: ', deuteron['electric_quadrupole'].values[0],'barn')
print('Energía de separación del protón:  ', deuteron['sp'].values[0]/1000,'+/-',deuteron['unc_sp'].values[0]/1000,'MeV')
print('Energía de separación del neutrón: ', deuteron['sn'].values[0]/1000,'+/-',deuteron['unc_sn'].values[0]/1000,'MeV')
print(dashes+dashes)

------------------------------------------------------------------------------------------------------------------------
Propiedades del deuterón:
A= 2 ; Z= 1 ( H ); N= 1
Año de descubrimiento:  1932
Vida media:  STABLE
Abundancia:  0.0145 %
Radio de carga:  2.1421 +/- 0.0088 fm
Energía de ligadura:  2.2245662000000004 +/- 0.0004 MeV
Masa:  2014.1017778439998 +/- 1.5000000000000002e-08 u.m.a. ; 1.8761239276950408 GeV
Dipolo magnético:  0.857438231 magnetones de Bohr
Cuadrupolo eléctrico:  0.0028578 barn
Energía de separación del protón:   2.2245662000000004 +/- 4.0000000000000003e-07 MeV
Energía de separación del neutrón:  2.2245662000000004 +/- 4.0000000000000003e-07 MeV
------------------------------------------------------------------------------------------------------------------------
